## Visualization utilities

In [6]:
import cv2 as cv
from cmath import sqrt
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import cv2
import threading
from selenium.webdriver.common.keys import Keys
import time

MARGIN = 10
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54)
def manos_centro(puntos,numero):
  return sqrt(((puntos[numero].x)-(puntos[0].x))**2+((puntos[numero].y)-(puntos[0].y))**2+((puntos[numero].y)-(puntos[0].y))**2).real

def draw_landmarks_on_image(rgb_image, detection_result,hilos):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN
       
    puntos=[]
    for landmark in hand_landmarks:
      puntos.append(landmark)
    actions = webdriver.ActionChains(driver)
    if not hilos[0] and manos_centro(puntos,8)<manos_centro(puntos,7) and manos_centro(puntos,12)<manos_centro(puntos,11) and manos_centro(puntos,16)<manos_centro(puntos,15) and manos_centro(puntos,20)<manos_centro(puntos,19):
        cv.putText(annotated_image, 'arranca',
                  (text_x, text_y), cv.FONT_HERSHEY_DUPLEX,
                  FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv.LINE_AA)
        hilos[0]=threading.Thread(target=movimiento,args=(actions,Keys.ARROW_UP,10000000,hilos,1))
        hilos[0].start()
    elif not hilos[1] and manos_centro(puntos,8)>manos_centro(puntos,7) and manos_centro(puntos,12)>manos_centro(puntos,11) and manos_centro(puntos,16)>manos_centro(puntos,15) and manos_centro(puntos,20)>manos_centro(puntos,19):
        cv.putText(annotated_image, 'derecha',
                  (text_x, text_y), cv.FONT_HERSHEY_DUPLEX,
                  FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv.LINE_AA)  
        hilos[1]=threading.Thread(target=movimiento,args=(actions,Keys.ARROW_RIGHT,0.01,hilos,1))
        hilos[1].start() 
    elif not hilos[2] and manos_centro(puntos,8)>manos_centro(puntos,7) and manos_centro(puntos,12)>manos_centro(puntos,11) and manos_centro(puntos,16)<manos_centro(puntos,15) and manos_centro(puntos,20)<manos_centro(puntos,19):
        cv.putText(annotated_image, 'izquierda',
                  (text_x, text_y), cv.FONT_HERSHEY_DUPLEX,
                  FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv.LINE_AA)
        hilos[2]=threading.Thread(target=movimiento,args=(actions,Keys.ARROW_LEFT,0.01,hilos,1))
        hilos[2].start()   
  

  return annotated_image


def movimiento(actions,key,time,lista,pos):
    actions.key_down(key).perform()
    time.sleep(time)
    actions.key_up(key).perform()
    lista[pos]=None

In [7]:
import argparse
from enum import Enum
from lib2to3.pgen2 import driver
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import time
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
GeckoDriverManager().install()
driver = webdriver.Firefox()
driver.get("https://mkpc.malahieude.net/mariokart.php")
html=driver.page_source
soup=BeautifulSoup(html,'html.parser')
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'fc-button fc-cta-consent fc-primary-button')]")))
elem = driver.find_element(By.XPATH, '//p[@class="fc-button-label"]').click()
time.sleep(2)
elem = driver.find_element(By.XPATH, '//input[@value="Grand Prix"]').click()
time.sleep(2)
elem = driver.find_element(By.XPATH, '//*[@id="perso-selector-mario"]').click()
time.sleep(2)
elem = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/img[1]').click()
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options, running_mode=vision.RunningMode.VIDEO, num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

cap = cv2.VideoCapture(0) 
timestamp = time.time()
hilos=[None,None,None]
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    image = frame
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_rgb)
    f_timestamp = round((time.time() - timestamp) * 1000)
    hand_landmarker_result = detector.detect_for_video(mp_image, f_timestamp)

    annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), hand_landmarker_result,hilos)
    hilos=[None,None,None]
    cv2.imshow('Hand Landmarks Detection', annotated_image[:, :, ::-1])
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
driver.close()
cap.release()
cv2.destroyAllWindows()






Exception in thread Thread-5713:
Traceback (most recent call last):
  File "c:\Users\david.fernandezrebor\AppData\Local\miniconda3\envs\ia\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\david.fernandezrebor\AppData\Local\miniconda3\envs\ia\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\david.fernandezrebor\AppData\Local\Temp\ipykernel_15784\3175463612.py", line 73, in movimiento
AttributeError: 'int' object has no attribute 'sleep'
Exception in thread Thread-5714:
Traceback (most recent call last):
  File "c:\Users\david.fernandezrebor\AppData\Local\miniconda3\envs\ia\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\david.fernandezrebor\AppData\Local\miniconda3\envs\ia\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\david.fernandezrebor\AppData\Local\Temp\ipykernel_15784\3175463612.py", line 73, in movimiento
AttributeE

InvalidSessionIdException: Message: Tried to run command without establishing a connection
